In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:0"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0.3):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(3, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 0.3
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                #- logprob.view(batchsize, nsamples)[:, :-1] 
                #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                #* positive_reinforcement_binary 
                logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 8) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
        

In [ ]:
train(epochs = 300000, npoints = 14, batchsize = 2000, nsamples = 8) #greedyprob - 8, without +ve reinforcement, 
#consider negative if performance > 0.3 + greedy

49.387001037597656 -13.966108322143555 -15.041854858398438 -13.793954849243164 -15.071525573730469 -15.071526527404785
49.2295036315918 -13.045592308044434 -14.818236351013184 -12.986042976379395 -14.488252639770508 -14.488251686096191
48.24250411987305 -12.33095645904541 -14.547760009765625 -12.349878311157227 -17.704345703125 -17.704347610473633
47.04650115966797 -11.350635528564453 -13.987713813781738 -11.781083106994629 -16.868501663208008 -16.868501663208008
46.59950256347656 -10.584723472595215 -13.500624656677246 -9.674517631530762 -13.599479675292969 -13.599479675292969
45.97200393676758 -9.496487617492676 -12.621761322021484 -10.490208625793457 -16.085405349731445 -16.085403442382812
45.849002838134766 -9.02286434173584 -12.244489669799805 -8.194128036499023 -11.014119148254395 -11.014118194580078
45.74250030517578 -9.13278579711914 -12.350801467895508 -8.57734203338623 -10.144916534423828 -10.144917488098145
45.84700393676758 -9.165037155151367 -12.318544387817383 -9.29298400

44.225502014160156 -6.43316650390625 -9.758466720581055 -5.758833885192871 -11.021692276000977 -11.021693229675293
44.208003997802734 -6.41860294342041 -9.747955322265625 -6.781064033508301 -8.196298599243164 -8.196300506591797
44.487003326416016 -6.4237141609191895 -9.745293617248535 -6.5499491691589355 -8.567496299743652 -8.567496299743652
44.21000289916992 -6.388223648071289 -9.735491752624512 -4.7497100830078125 -11.073869705200195 -11.073867797851562
44.11100387573242 -6.3526811599731445 -9.731667518615723 -5.666909694671631 -9.387958526611328 -9.387958526611328
44.402503967285156 -6.357096195220947 -9.710107803344727 -7.10239839553833 -7.041673183441162 -7.0416741371154785
44.26250076293945 -6.444427013397217 -9.812846183776855 -7.245406150817871 -8.199718475341797 -8.199718475341797
44.19550323486328 -6.578996658325195 -9.98338794708252 -6.692519187927246 -8.366338729858398 -8.366338729858398
44.0420036315918 -6.66404914855957 -10.102957725524902 -6.50602388381958 -10.1514463424

43.92000198364258 -8.593796730041504 -12.336244583129883 -5.801205158233643 -12.302968978881836 -12.302968978881836
43.7660026550293 -8.562103271484375 -12.193000793457031 -7.690871238708496 -13.419296264648438 -13.419296264648438
43.78600311279297 -8.299697875976562 -11.97134017944336 -8.875242233276367 -14.259017944335938 -14.259017944335938
43.61650085449219 -8.062750816345215 -11.737663269042969 -9.252535820007324 -12.63653564453125 -12.636536598205566
43.779502868652344 -7.683599948883057 -11.34245777130127 -7.503855228424072 -11.929204940795898 -11.929203987121582
43.812503814697266 -7.406938552856445 -11.125571250915527 -7.041316032409668 -10.688202857971191 -10.688202857971191
43.70050048828125 -7.270053386688232 -10.980546951293945 -6.5353193283081055 -9.745275497436523 -9.74527645111084
43.63600158691406 -7.298203468322754 -10.998997688293457 -8.999307632446289 -13.988471984863281 -13.988471984863281
43.523502349853516 -7.407841682434082 -11.149847984313965 -7.558918952941894

43.220001220703125 -8.123150825500488 -12.070090293884277 -7.01393985748291 -10.360685348510742 -10.360684394836426
43.369503021240234 -7.972178936004639 -12.051895141601562 -7.840106964111328 -11.833248138427734 -11.833248138427734
43.16850280761719 -7.954246520996094 -12.037281036376953 -7.3035502433776855 -9.297798156738281 -9.297798156738281
43.185001373291016 -8.104775428771973 -12.31844711303711 -7.492983341217041 -10.963151931762695 -10.963150024414062
43.220001220703125 -8.436661720275879 -12.558642387390137 -8.920265197753906 -14.780171394348145 -14.780171394348145
42.9530029296875 -8.622431755065918 -12.614407539367676 -9.152259826660156 -10.220664978027344 -10.220664978027344
42.83650207519531 -8.786365509033203 -12.641704559326172 -8.216569900512695 -17.42437744140625 -17.42437744140625
43.16950225830078 -8.830548286437988 -12.627053260803223 -9.59676742553711 -9.076421737670898 -9.076422691345215
43.02100372314453 -8.52951431274414 -12.302780151367188 -9.08399772644043 -15

42.75600051879883 -8.130651473999023 -12.28835678100586 -7.992207050323486 -9.64588737487793 -9.645886421203613
42.488502502441406 -8.141484260559082 -12.323132514953613 -9.665029525756836 -15.489416122436523 -15.48941421508789
42.724002838134766 -8.214844703674316 -12.290651321411133 -7.753567695617676 -13.103787422180176 -13.103787422180176
42.830501556396484 -8.251703262329102 -12.338008880615234 -7.680445194244385 -14.153806686401367 -14.153806686401367
42.86250305175781 -8.314217567443848 -12.377965927124023 -7.398051738739014 -11.412351608276367 -11.412351608276367
42.86150360107422 -8.220203399658203 -12.301706314086914 -7.06776237487793 -14.084729194641113 -14.084729194641113
42.94350051879883 -8.139756202697754 -12.205243110656738 -7.0087385177612305 -10.900150299072266 -10.900150299072266
42.72600173950195 -8.14022445678711 -12.200644493103027 -6.797794342041016 -11.31041431427002 -11.310413360595703
42.676002502441406 -8.031286239624023 -12.146369934082031 -7.58788537979126 

...

40.55950164794922 -5.6194844245910645 -8.492048263549805 -8.779365539550781 -12.010980606079102 -12.010981559753418
40.345001220703125 -5.586676120758057 -8.430044174194336 -9.467425346374512 -13.331460952758789 -13.331460952758789
40.58100128173828 -5.699478626251221 -8.479612350463867 -1.492472767829895 -16.160398483276367 -16.160398483276367
40.557003021240234 -5.641399383544922 -8.575943946838379 -9.636436462402344 -11.577835083007812 -11.577834129333496
40.551002502441406 -5.688382148742676 -8.393533706665039 -1.5128355026245117 -4.277712821960449 -4.277713298797607
40.45650100708008 -5.513563632965088 -8.44411849975586 -7.897443771362305 -13.256197929382324 -13.256197929382324
40.58250045776367 -5.454792022705078 -8.450071334838867 -7.356788635253906 -1.4518769979476929 -1.4518767595291138
40.4640007019043 -5.49285364151001 -8.403837203979492 -1.001186490058899 -2.4701638221740723 -2.470165252685547
40.337501525878906 -5.397047996520996 -8.423555374145508 -7.92388916015625 -12.85

40.32050323486328 -5.761327266693115 -8.951237678527832 -2.8365087509155273 -10.25656509399414 -10.25656509399414
40.30550003051758 -6.057490348815918 -8.93431282043457 -9.445303916931152 -0.7392022609710693 -0.7392017841339111
40.30650329589844 -6.079028129577637 -9.027604103088379 -0.39902690052986145 -13.006510734558105 -13.006511688232422
40.54500198364258 -5.975649833679199 -8.862648010253906 -1.092766284942627 -3.007218837738037 -3.007218837738037
40.40850067138672 -5.856452465057373 -8.903719902038574 -8.342144012451172 -10.376266479492188 -10.376266479492188
40.41050338745117 -5.946643829345703 -8.831273078918457 -9.678962707519531 -15.260082244873047 -15.260082244873047
40.444000244140625 -5.841836452484131 -8.83896541595459 -9.558368682861328 -16.755407333374023 -16.755407333374023
40.59300231933594 -5.782405376434326 -8.730146408081055 -1.8366336822509766 -11.527650833129883 -11.527652740478516
40.37750244140625 -5.655065536499023 -8.688467979431152 -0.7243180274963379 -4.03

40.5150032043457 -5.599238395690918 -8.448751449584961 -7.7909135818481445 -0.615936279296875 -0.6159353256225586
40.33150100708008 -5.531646728515625 -8.38160228729248 -0.7283709049224854 -2.1841259002685547 -2.184126377105713
40.45450210571289 -5.51209020614624 -8.473855018615723 -0.4394221305847168 -3.038679599761963 -3.0386805534362793
40.53150177001953 -5.599704265594482 -8.685833930969238 -8.106376647949219 -11.572711944580078 -11.572712898254395
40.361000061035156 -5.709327220916748 -8.693206787109375 -0.3413419723510742 -0.7642548084259033 -0.7642556428909302
40.264503479003906 -5.751819610595703 -8.891809463500977 -9.179893493652344 -16.16451072692871 -16.16451072692871
40.40950012207031 -5.820950508117676 -8.975707054138184 -0.8034433126449585 -0.4479541778564453 -0.44795334339141846
40.422000885009766 -5.867993354797363 -8.964171409606934 -9.079076766967773 -12.79416275024414 -12.79416275024414
40.54100036621094 -6.03851318359375 -9.115138053894043 -9.024208068847656 -13.287

40.46950149536133 -5.94699764251709 -9.034839630126953 -1.1843550205230713 -1.559035301208496 -1.5590357780456543
40.62600326538086 -5.923888206481934 -8.934060096740723 -7.262272834777832 -12.832965850830078 -12.832965850830078
40.404502868652344 -5.995519638061523 -8.9522123336792 -9.686762809753418 -0.04831710457801819 -0.048315078020095825
40.50550079345703 -5.926607608795166 -9.015349388122559 -0.9520163536071777 -2.9587278366088867 -2.958728313446045
40.400001525878906 -5.834110736846924 -9.041682243347168 -9.184422492980957 -12.514474868774414 -12.51447582244873
40.42250061035156 -6.102734088897705 -9.19461727142334 -7.063628196716309 -11.921067237854004 -11.92106819152832
40.371002197265625 -6.174692153930664 -9.231552124023438 -8.092918395996094 -18.294761657714844 -18.294761657714844
40.250003814697266 -6.360418319702148 -9.428933143615723 -7.361339569091797 -0.7443282604217529 -0.7443273067474365
40.509002685546875 -6.249654769897461 -9.329231262207031 -10.54052448272705 -9.

40.730003356933594 -5.9907941818237305 -8.929020881652832 -1.0434684753417969 -11.089005470275879 -11.089005470275879
40.58350372314453 -5.947437763214111 -8.943055152893066 -5.753781318664551 -10.405686378479004 -10.405686378479004
40.46800231933594 -5.899402618408203 -8.847241401672363 -7.976578712463379 -0.44171833992004395 -0.44171977043151855
40.60850143432617 -5.9349188804626465 -8.820479393005371 -8.375152587890625 -12.383892059326172 -12.383892059326172
40.596500396728516 -5.953625679016113 -8.935220718383789 -2.0226893424987793 -12.44015884399414 -12.440159797668457
40.45750045776367 -5.815223217010498 -8.74853801727295 -8.034490585327148 -9.470359802246094 -9.47036075592041
40.437503814697266 -5.517485618591309 -8.778519630432129 -1.598897933959961 -12.287721633911133 -12.287721633911133
40.481502532958984 -5.7287702560424805 -8.846190452575684 -1.3837890625 -16.152610778808594 -16.152610778808594
40.355003356933594 -5.766768932342529 -8.886194229125977 -8.1473388671875 -14.5